# Generate daily VWAP using dedicated service - C#

### Overview
The Volume Weighted Average price is similar to a moving average, except volume is included to weight the average price over a one trading day period.

It serves as a reference point for one trading day prices. Comparing current prices with the VWAP values enables to:
* mesure trading efficiency by comparing individual trades to VWAP
* highlight liquidity points as vwap reflects prices levels weighted by volume
* determine the intraday trend
* etc.

This sample demonstrates how to request and plot from a dedicated data service **on-demand** `daily` VWAP price.

### Inputs/outputs
Daily VWAP indicator requires instrument's identifier, date time intervals and time range as per inputs and returns the daily VWAP data points.

VWAP is a cumulative indicator which means the number of data points progressively increases throughtout the day.

This sample shows how to plot a simple daily vwap for basis technical analysis using an open source library. 

### Services used
This sample uses gRPC requests in order to retrieve VWAP indicator from the hosted service. The queried endpoint in this script are:
* *DailyVwapsService*: to directly retrieve daily VWAP objects from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Daily.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package

***

# Run daily VWAP sample

### Step 1: Install packages

In [1]:
#i "nuget: file://~/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 0.*-pre*"
#r "nuget: XPlot.Plotly.Interactive"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Restore sources file://~/.nuget/packages/ https://api.nuget.org/v3/index.json Installed Packages Systemathics.Apis, 0.13.2-pre XPlot.Plotly.Interactive, 4.0.6

Loading extensions from `XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

In [2]:
using Systemathics.Apis.Helpers;
using Systemathics.Apis.Type.Shared.V1;
using Systemathics.Apis.Services.Daily.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Net.Client;
using Grpc.Core;
using XPlot.Plotly;
using XPlot.Plotly.Interactive;

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [3]:
// Get token as metadata
var headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
var channel = ChannelHelpers.GetChannel();

### Step 3: Create and process request

#### 3.1 Request Parameters
To request *daily vwap* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Service called: *DailyVwapsService* in order to retrieve daily data

In [4]:
// Set instrument
var ticker = "ACA";
var exchange = "XPAR";

#### 3.2 Request creation
The following code snippet creates *gRPC client*, process request and returns request reply.

In [5]:
// Instantiate the daily vwap service
var service = new DailyVwapsService.DailyVwapsServiceClient(channel);

// Create and process daily vwap request
var request = new DailyVwapsRequest 
{ 
    Identifier = new Identifier { Exchange = exchange, Ticker = ticker}
};
var reply = service.DailyVwaps(request, headers);

The following code snippet displays the *daily vwap* data points:

In [6]:
display(reply.Data)

index Date Price Volume Score 0 Year Month Day 2020 10 26 7.385 5742545 0 1 Year Month Day 2020 10 27 7.1299 7725116 0 2 Year Month Day 2020 10 28 6.7162 11676944 0 3 Year Month Day 2020 10 29 6.6424 8725304 0 4 Year Month Day 2020 10 30 6.7282 6559964 0 5 Year Month Day 2020 11 2 6.9343 5612951 0 6 Year Month Day 2020 11 3 7.2903 12646540 0 7 Year Month Day 2020 11 4 7.3554 7999349 0 8 Year Month Day 2020 11 5 7.4724 7188295 0 9 Year Month Day 2020 11 6 7.4893 6323426 0 10 Year Month Day 2020 11 9 8.2196 19934709 0 11 Year Month Day 2020 11 10 8.7428 19264718 0 12 Year Month Day 2020 11 11 8.7795 8502072 0 13 Year Month Day 2020 11 12 8.4918 8483692 0 14 Year Month Day 2020 11 13 8.5986 10049235 0 15 Year Month Day 2020 11 16 8.8412 12781132 0 16 Year Month Day 2020 11 17 8.7522 8032282 0 17 Year Month Day 2020 11 18 8.8949 7221409 0 18 Year Month Day 2020 11 19 8.8312 6998034 0 19 Year Month Day 2020 11 20 8.9634 7467662 0 (494 more)

### Step 4: Visualize data

#### 4.1 Plot vwap and volume
The following code snippets retrive *Close prices* over the look back period from the request result and plot them with the package of your choice.<br><i>`XPlot.Plotly`</i> is used as per open source display package.

In [7]:
// Extract the daily vwap and the volume
var vwaps = reply.Data.Select(t => new Tuple<DateTime, double>(new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Price));
var volumes = reply.Data.Select(t => new Tuple<DateTime, double>(new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Volume));

In [8]:
// Create the graph
var vwapGraph = new List<Scattergl>
{
    new Scattergl
    {
        name = "Vwap",
        mode = "lines",
        x = vwaps.Select(c => c.Item1),
        y = vwaps.Select(c => c.Item2)
    },
    new Scattergl
    {
        name = "Volume",
        mode = "bar",
        x = volumes.Select(v => v.Item1),
        y = volumes.Select(v => v.Item2),
        yaxis = "y2"
    } 
};


var layout = new Layout.Layout{ 
    title = $"Vwap & Volume for {ticker}",
    yaxis = new Yaxis {
        title = "Price", 
        autorange = true,
        side = "left",
    },
    yaxis2 = new Yaxis {
        title = "Volume", 
        autorange = true,
        side = "right",  
    },
};

var chart = Chart.Plot(vwapGraph, layout);
chart.WithXTitle($"Time");
chart.Width = 1500;
chart.Height = 800;
display(chart);

<!DOCTYPE html>